<a href="https://colab.research.google.com/github/pitapatat/Data_Scraping/blob/main/%5B%ED%81%AC%EB%A1%A4%EB%A7%81%5D__%EC%98%AC%EB%A6%AC%EB%B8%8C%EC%98%81_%EC%84%B8%EC%9D%BC%EC%83%81%ED%92%88(beautifulsoup).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 라이브러리

In [ ]:
import requests
from bs4 import BeautifulSoup
import time 
import json

### 홈페이지 html parsing

In [ ]:
page = 1 
url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=10000010001&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리
raw = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(raw.text, 'html.parser')
#print(html)

In [ ]:
total_num = html.select('p.cate_info_tx span')[0].text.strip() #전체 세일 등록 상품 개수

In [ ]:
last_page = int(total_num) // 48 + 1
print(last_page)
#page = range(1,last_page+1)

17


In [ ]:
# 상품 카테고리 
category = html.select('div#Container div.common-menu li')[1:19]
key = category[0].text
value = category[0].find('button')['data-ref-dispcatno']
print(key, value)

print(len(category))
category[1].text

스킨케어 10000010001
18


'마스크팩'

In [ ]:
## 전체 카테고리 넘버링
# for i in range(len(category)):
#     value = category[i].find('button')['data-ref-dispcatno']  # 카테고리 넘버
#     key = category[i].text   #카테고리명
#     print(key, value)

In [ ]:
products = html.select('ul.cate_prd_list.gtm_sale_list div.prd_info')
#print(len(products))

# 예시- 첫번째 상품
products[0].select('div.prd_name span')[0].text  # 회사명
products[0].select('div.prd_name p')[0].text  # 상품명
products[0].select('p.prd_price')[0].text.split()  # 정가, 세일가
#products[0].select('a.prd_thumb.goodsList')[0]['data-ref-itemno']  #상품 카테고리 
products[0].select('a')[0]['href']  # 상품 url 
# 전체 상품
# results = []
# for i in range(len(products)):
#     result = []
#     company = products[i].select('div.prd_name span')[0].text
#     product_name = products[0].select('div.prd_name p')[0].text
#     fixed_price , sale_price = products[0].select('p.prd_price')[0].text.split()

#     contents = {'company': company, 'product_name': product_name, 
#                 'price' : fixed_price, 'sale_price' : sale_price}
#     result.append(contents)

# results.extend(result)

'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000163539&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'

In [ ]:
url = products[0].select('a')[0]['href'] 

goods = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(goods.text, 'html.parser')

cat = html.select('ul.loc_history')[0].select('a.cate_y')
cat_1 = cat[0].text
cat_2 = cat[1].text
cat_3 = cat[2].text
print(cat_1, cat_2, cat_3)  #카테고리 대/중/소 분류

스킨케어 에센스/크림 에센스/세럼


### 크롤링 함수
- 올리브영 세일 > 상품명, 카테고리, 정가/세일가, 세부페이지 url 

In [ ]:
def get_sale_products(url):        
    raw = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, 'html.parser')

    # 전체 세일 상품 갯수 -> 마지막 페이지 수 구하기
    total_num = html.select('p.cate_info_tx span')[0].text.strip().replace(',','')
    page_num = int(total_num) // 48 + 1
    print(f"last_page_number: {page_num}")


    # 전체 페이지에 대한 세일 상품 추출
    results = []
    for i in range(1, page_num):
        result = []

        for i in range(len(products)):           
            # 회사명
            company = products[i].select('div.prd_name span')[0].text
            # 상품명
            product_name = products[i].select('div.prd_name p')[0].text
            # 가격(정가, 세일가)
            fixed_price , sale_price = products[i].select('p.prd_price')[0].text.split()

            # 상품 url (세부 정보 추출)
            url = products[i].select('a')[0]['href']

            # 상품 세부 카테고리 --------- 추가하면 런타임이 너무 길어짐
            # goods = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
            # html = BeautifulSoup(goods.text, 'html.parser')

            # cat = html.select('ul.loc_history')[0].select('a.cate_y')
            # cat_1 = cat[0].text
            # cat_2 = cat[1].text
            # cat_3 = cat[2].text

            contents = {'company': company, 'product_name': product_name, 
                        'price' : fixed_price, 'sale_price' : sale_price, 'url' : url}
            
            result.append(contents)

        results.extend(result)
        
        # 어떤 데이터를 보낼까? 
        #text = results[0]
        ## 카카오톡 알림
        #send_to_kakao(text)
    
    return results


### 카카오톡 알림 설정( 다시 ... 오류..)
https://www.youtube.com/watch?v=gg2sjQ0877U

In [ ]:
# KAKAO_TOKEN = '2d9d00cc5dbd4174e663ee2de28a275b'

# def send_to_kakao(text):
#     header = {'Authorization':'Bearer'+ KAKAO_TOKEN}
#     url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
#     post = {"object_type": text,
#         "text": "텍스트 영역입니다. 최대 200자 표시 가능합니다.",
#         "link": {
#             "web_url": "https://developers.kakao.com",
#             "mobile_web_url": "https://developers.kakao.com"
#         },
#         }
#     data = {'template_object': json.dumps(post)}
#     requests.post(url, headers = header, data = data)

### 크롤링 결과 데이터 프레임으로 만들기 

In [ ]:

from typing import Dict
import pandas as pd

def to_csv(data: Dict):
    df = pd.DataFrame(data).to_csv('/content/sample_data/olive_young_sale.csv', index = False)
    return df

### 함수 실행 결과

In [ ]:
page = 1 
#url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=10000010001&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리(스킨)
url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=24&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리(전체)
result = get_sale_products(url)
print(result)
print(len(result))

last_page_number: 164
[{'company': '아이소이', 'product_name': '[한정기획] 아이소이 잡티세럼 업 40ml 기획 (+15ml)', 'price': '54,000원', 'sale_price': '36,800원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000163539&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '닥터지', 'product_name': '[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)', 'price': '36,000원', 'sale_price': '25,200원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000114129&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '아누아', 'product_name': '아누아 어성초 77 수딩 토너 350ml 기획 (+40ml +앰플10ml)', 'price': '30,500원', 'sale_price': '22,800원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000162254&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '에스트라', 'product_name': '[포토카드 증정]에스트라 에이시카365 흔적 진정 세럼 40ml 기획(세럼5ml+크림10ml)', 'price': '38,000원', 'sale_price': '28,500원

In [ ]:
df = to_csv(result)
df = pd.read_csv('/content/sample_data/olive_young_sale.csv')
df.head(3)

,company,product_name,price,sale_price,url
0,아이소이,[한정기획] 아이소이 잡티세럼 업 40ml 기획 (+15ml),"54,000원","36,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
1,닥터지,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"36,000원","25,200원",https://www.oliveyoung.co.kr/store/goods/getGo...
2,아누아,아누아 어성초 77 수딩 토너 350ml 기획 (+40ml +앰플10ml),"30,500원","22,800원",https://www.oliveyoung.co.kr/store/goods/getGo...


### 카톡 알림 